In [0]:
from azure.storage.blob import BlobServiceClient
from pyspark.sql.functions import col, decode, split, element_at,udf,input_file_name,regexp_extract,expr
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, LongType


In [0]:
# import time

# time.sleep(600)

In [0]:
config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()

keyvault_name = f"ingest{lz_key}-meta002-{env}"

In [0]:
# Set up Service Principal credentials in Spark
client_secret = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-CLIENT-SECRET')
tenant_id = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-TENANT-ID')
client_id = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-CLIENT-ID')
 
curated_storage_account = f"ingest{lz_key}curated{env}"
xcutting_storage_account = f"ingest{lz_key}xcutting{env}"

storage_accounts = [curated_storage_account,xcutting_storage_account]

for storage_account in storage_accounts:
    configs = {
            f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net": "OAuth",
            f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net":
                "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
            f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net": client_id,
            f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net": client_secret,
            f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net":
                f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
        }
    for key,val in configs.items():
        spark.conf.set(key,val)

In [0]:

expected_schema = StructType([
    StructField("operation", StringType(), True),
    StructField("transaction_id", StringType(), True),
    StructField("relation_id", StringType(), True),
    StructField("a360_record_id", StringType(), True),
    StructField("process_time", TimestampType(), True),
    StructField("status", IntegerType(), True),
    StructField("input", StringType(), True),  # Contains nested JSON as a string
    StructField("exception_description", StringType(), True),
    StructField("error_status", StringType(), True),
    StructField("a360_file_id", StringType(), True),
    StructField("file_size", LongType(), True),
    StructField("s_md5", StringType(), True),
    StructField("s_sha256", StringType(), True),
    StructField("timestamp", TimestampType(), True),  # may be used as process_time
    StructField("filename", StringType(), True),
    StructField("submission_folder", StringType(), True),
    StructField("file_hash", StringType(), True)
])

In [0]:
ARM_segment = "FTADEV" if env == "sbox" else "FTA"
ARIA_segment = "APLFTA"

In [0]:
#### Set up Auto Loader job
from pyspark.sql.functions import input_file_name,regexp_extract,col,expr


sas_token = dbutils.secrets.get(scope=f"ingest{lz_key}-meta002-{env}", key=f"ARIA{ARM_segment}-SAS-TOKEN")
storage_account_name = "a360c2x2555dz"
container_name = "dropzone"
sub_dir = f"ARIA{ARM_segment}/response"

input_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/{sub_dir}"

spark.conf.set(
    f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net",
    sas_token
)

schema_location = f"/mnt/autoLoaderSchema/ARM{ARIA_segment}/response/read_stream"

# Define a file regex that matches files ending with .rsp
file_regex = ".*\\.rsp$"

output_container_name = "silver" 

output_storage_account_name = f"ingest{lz_key}curated{env}"

output_subdir_amalgamated_responses = f"ARIADM/ARM/response/{ARIA_segment}/amalgamated_responses"
amalgamated_responses_path = f"wasbs://{output_container_name}@{output_storage_account_name}.blob.core.windows.net/{output_subdir_amalgamated_responses}"

output_subdir_input_upload = f"ARIADM/ARM/response/{ARIA_segment}/input_upload"
input_upload_responses_path = f"wasbs://{output_container_name}@{output_storage_account_name}.blob.core.windows.net/{output_subdir_input_upload}"


output_subdir_create_record_upload = f"ARIADM/ARM/response/{ARIA_segment}/create_record"
create_record_responses_path = f"wasbs://{output_container_name}@{output_storage_account_name}.blob.core.windows.net/{output_subdir_create_record_upload}"


output_subdir_upload_file_upload = f"ARIADM/ARM/response/{ARIA_segment}/upload_file"
upload_file_responses_path = f"wasbs://{output_container_name}@{output_storage_account_name}.blob.core.windows.net/{output_subdir_upload_file_upload}"

In [0]:
check_point_path = f"abfss://db-rsp-checkpoint@ingest{lz_key}xcutting{env}.dfs.core.windows.net/ARM{ARIA_segment}/RSP/"

schema_location = f"abfss://db-rsp-checkpoint@ingest{lz_key}xcutting{env}.dfs.core.windows.net/ARM{ARIA_segment}/RSP/schema"

In [0]:
### Run autoloader read stream

df = (spark.readStream.format("cloudFiles")
      .schema(expected_schema)
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", schema_location)
    .option("multiline", "true")
    .option("cloudFiles.schemaEvolutionMode", "none")
    .option("checkpointLocation", f"{check_point_path}/rsp_readStream")
    .load(input_path)
    .select( "*",col("_metadata.file_path").alias("_file_path"),
             col("_metadata.file_modification_time").alias("_file_modification_time")
)
    # .withColumn("file_name", regexp_extract(input_file_name(),"response\/(.*)",1))
    .withColumn("id", expr("uuid()")))

In [0]:
df_cr = df.filter(col("operation")=="create_record")
df_iu = df.filter(col("operation")=="input_upload")
df_uf = df.filter(col("operation")=="upload_new_file")

In [0]:
### 4 write streams to Silver container in blob storage

output_sas = dbutils.secrets.get(scope=keyvault_name, key=f"CURATED-{env}-SAS-TOKEN")

spark.conf.set(
    f"fs.azure.sas.{output_container_name}.{output_storage_account_name}.blob.core.windows.net",
    output_sas
)

### first save the complete table

df_complete = df.writeStream \
    .format("delta")\
        .option("checkpointLocation", f"{check_point_path}/amalgamated")\
            .outputMode("append")\
                    .start(amalgamated_responses_path)

In [0]:
### Input upload table

df_input_upload = df_iu.select("id","operation","timestamp","status","exception_description","error_status","filename","submission_folder","file_hash","_file_path","_file_modification_time"
)\
    .writeStream \
    .format("delta")\
        .option("checkpointLocation", f"{check_point_path}/input_upload")\
            .outputMode("append")\
                .start(input_upload_responses_path)

In [0]:
config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env_name = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()

print(f"env_code: {lz_key}")  # This won't be redacted
print(f"env_name: {env_name}")  # This won't be redacted

KeyVault_name = f"ingest{lz_key}-meta002-{env_name}"
print(f"KeyVault_name: {KeyVault_name}") 

In [0]:
### Create Record table
df_create_record = df_cr.select("id","operation","transaction_id","relation_id","a360_record_id","process_time","status","input","exception_description","error_status","_file_path","_file_modification_time"
).writeStream \
    .format("delta")\
        .option("checkpointLocation", f"{check_point_path}/create_record")\
            .outputMode("append")\
                .start(create_record_responses_path)

In [0]:
### upload file table
df_upload_file = df_uf.select("id","operation","transaction_id","relation_id","a360_record_id","process_time","status","input","exception_description","error_status","a360_file_id","file_size","s_md5","s_sha256","_file_path","_file_modification_time"
)\
    .writeStream \
    .format("delta")\
        .option("checkpointLocation", f"{check_point_path}/upload_file")\
            .outputMode("append")\
                .start(upload_file_responses_path)

In [0]:
Wait 15 minutes for any stream to stop itself, or auto stop after 15mins.

try:
    spark.streams.awaitAnyTermination(900)
finally:
    # Stop all streams
    df_complete.stop()
    df_input_upload.stop()
    df_create_record.stop()
    df_upload_file.stop()
    print("All streams have been stopped")


df_input_upload = spark.read.format("delta").load(input_upload_responses_path)
 
# # Read the response data for create_record_upload
df_create_record_upload = spark.read.format("delta").load(create_record_responses_path)
 
# # Read the response data for upload_file
df_upload_file_upload = spark.read.format("delta").load(upload_file_responses_path) 
 
# Read the response data for df_amalgamated_responses
df_amalgamated_responses = spark.read.format("delta").load(amalgamated_responses_path)

print(f"Records in df_input_upload: {df_input_upload.count()}")
print(f"Records in df_create_record_upload: {df_create_record_upload.count()}")
print(f"Records in df_upload_file_upload: {df_upload_file_upload.count()}")
print(f"Records in df_amalgamated_responses: {df_amalgamated_responses.count()}")

In [0]:
dbutils.notebook.exit("Notebook completed successfully")